In [6]:
import sys
sys.path.append("../script/")

import Functions
from Engine import Engine
from GTGP import GTGP


from sklearn.metrics import classification_report,accuracy_score,confusion_matrix,f1_score
from sklearn.model_selection import train_test_split
import numpy as np
import importlib
from time import time
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
import numpy as np
import pandas as pd
from time import time

In [2]:
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.utils import shuffle
def plot_roc_curve(true_y, y_prob):
    """
    plots the roc curve based of the probabilities
    """
    
    fpr, tpr, thresholds = roc_curve(true_y, y_prob)
    plt.plot(fpr, tpr)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    print(roc_auc_score(true_y,prob[:,1]))

# data

In [7]:
df = pd.read_csv("../data/shuttle.dat",delimiter=',')
X = df.iloc[:,:-1].to_numpy().astype("float")
y = df.iloc[:,-1].to_numpy().astype("int")

seeds = [10086, 200,500,30506,30405,30420,10056,7059,40965,5398,869543,83491,823190,
         48392,2810,48392,3498210,483902,859032,12890,538920,86954,54309,6504,9840,
         219805,548,2981,432890,5438908,219094,5843902,60854,979,12890,2108,4093]
train_size = 0.7
dataset = 'shuttle'

In [8]:
def to_one_hot(y,yt):
    le = LabelEncoder()
    ohe = OneHotEncoder()
    y_train = le.fit_transform(y)
    y_one_hot = ohe.fit_transform(y_train.reshape(-1,1))
    
    y_test = le.transform(yt)
    yt_one_hot = ohe.transform(y_test.reshape(-1,1))
    
    return y_train,y_test,y_one_hot,yt_one_hot
y_train,y_test,y_train_one_hot,y_test_one_hot = to_one_hot(y_train,y_test)

# model

In [29]:
def fit_trees():
    learning_rate=0.1
    max_depth=3
    bins=8
    lam=100

    gtgp = GTGP(learning_rate=learning_rate,max_depth=max_depth,bins=bins,lam=lam)

    total_size=10
    elite_size = 10
    epoch= 300
    gp_epoch= 3
    verbose = 1
    tolerance=0.1

    gtgp.fit(X_train,y_train,total_size=total_size,elite_size = elite_size,epoch=epoch,gp_epoch=gp_epoch,tolerance=tolerance,verbose=verbose)

    retrain_epoch=20
    alpha=0
    beta=1
    gammer=0

    verbose=1
    gtgp.lam = 10
    gtgp.retrain_estimators(X_test,y_test,retrain_epoch=retrain_epoch,alpha=alpha,beta=beta,gammer=gammer,verbose=verbose)

    return gtgp

for i in range(30):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
    y_train,y_test,y_train_one_hot,y_test_one_hot = to_one_hot(y_train,y_test)
    gtgp = fit_trees()

    print(classification_report(y_train,np.argmax(gtgp.train_p,axis=1),digits=5))
    print(classification_report(y_test,np.argmax(gtgp.test_p,axis=1),digits=5))

    num_trees,depth,num_nodes = gtgp.print_model()

    train_acc = accuracy_score(y_train,np.argmax(gtgp.train_p,axis=1))
    test_acc = accuracy_score(y_test,np.argmax(gtgp.test_p,axis=1))
    train_f1 = f1_score(y_train,np.argmax(gtgp.train_p,axis=1),average='macro')
    test_f1 = f1_score(y_test,np.argmax(gtgp.test_p,axis=1),average='macro')


    with open('./benchmark/'+dataset+'.csv','a') as f:
        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)

15 59 1
19 79 2
26 110 3
33 137 4
35 151 5
36 150 6
39 155 7
42 172 8
51 231 9
50 222 10
56 256 11
57 261 12
61 279 13
63 291 14
64 296 15
65 299 16
66 300 17
67 303 18
67 299 19
68 302 20
68 298 21
68 296 22
68 296 23
68 296 24
70 306 25
73 327 26
75 335 27
76 340 28
77 345 29
79 353 30
81 361 31
82 364 32
82 362 33
82 362 34
82 362 35
85 381 36
85 381 37
87 395 38
89 415 39
90 426 40
91 435 41
91 433 42
92 438 43
92 438 44
92 432 45
93 441 46
95 447 47
95 441 48
95 441 49
95 441 50
96 452 51
97 457 52
98 464 53
98 464 54
99 469 55
100 458 56
102 468 57
102 468 58
104 478 59
104 478 60
105 485 61
106 494 62
107 499 63
107 495 64
107 495 65
107 495 66
107 495 67
111 517 68
111 517 69
111 515 70
112 520 71
112 520 72
113 527 73
113 527 74
114 532 75
114 532 76
114 532 77
114 532 78
115 537 79
115 537 80
115 537 81
115 537 82
115 537 83
116 542 84
116 542 85
116 542 86
118 558 87
119 569 88
119 569 89
119 569 90
119 569 91
119 569 92
119 569 93
119 569 94
120 576 95
120 576 96
121 583 97

In [39]:
learning_rate=0.1
max_depth=4
bins=8
lam=1

gtgp = GTGP(learning_rate=learning_rate,max_depth=max_depth,bins=bins,lam=lam)

total_size=10
elite_size = 10
epoch=2000
gp_epoch=3
verbose = 1
re_train_epoch = 20
tolerance=0.3

gtgp.fit(X_train,y_train,total_size=total_size,elite_size = elite_size,epoch=epoch,gp_epoch=gp_epoch,tolerance=tolerance,verbose=verbose)

5 21 1
7 29 2
8 34 3
8 32 4
9 41 5
11 45 6
11 43 7
13 49 8
15 59 9
15 55 10
18 80 11
20 80 12
19 65 13
19 65 14
20 70 15
22 80 16
22 76 17
23 89 18
23 89 19
23 81 20
23 81 21
23 81 22
23 81 23
23 79 24
24 84 25
24 84 26
24 84 27
24 84 28
25 89 29
25 89 30
25 89 31
25 87 32
25 87 33
27 103 34
27 103 35
28 114 36
28 106 37
28 106 38
29 109 39
29 109 40
29 109 41
30 114 42
30 114 43
30 112 44
31 121 45
31 121 46
31 121 47
31 121 48
31 121 49
31 121 50
31 121 51
31 121 52
31 121 53
32 132 54
32 132 55
32 124 56
32 118 57
32 116 58
32 116 59
33 121 60
33 121 61
34 126 62
34 126 63
35 133 64
35 133 65
35 133 66
35 133 67
35 133 68
35 133 69
35 133 70
35 133 71
35 129 72
35 129 73
36 138 74
36 138 75
36 138 76
36 138 77
36 138 78
36 138 79
36 136 80
36 136 81
36 136 82
36 136 83
36 136 84
36 136 85
37 145 86
37 145 87
37 145 88
37 145 89
37 145 90
37 143 91
37 143 92
38 148 93
38 148 94
38 146 95
38 146 96
38 146 97
38 146 98
38 146 99
39 151 100
39 151 101
39 151 102
40 158 103
41 169 104
42

In [53]:
retrain_epoch=300
alpha=1
beta=1
gammer=0

verbose=1
gtgp.lam = 10
train_acc,test_acc,train_sse,test_sse = gtgp.retrain_estimators(X_test,y_test,retrain_epoch=retrain_epoch,alpha=alpha,beta=beta,gammer=gammer,verbose=verbose)


# verbose=0
# gtgp.retrain_estimators(retrain_epoch=retrain_epoch,alpha=alpha,beta=beta,gammer=gammer,verbose=verbose)

retrain  1 :
	train: 0.9991132568106803 295.4084585469799 	test: 0.9988505747126437 131.12269430655536
retrain  2 :
	train: 0.9996551554263757 57.06304374211339 	test: 0.9994252873563219 28.576574566456067
retrain  3 :
	train: 0.99977831420267 31.338611854769926 	test: 0.9994827586206897 17.777814306128256
retrain  4 :
	train: 0.999802945957929 22.9194231341963 	test: 0.9995402298850574 14.37003832723499
retrain  5 :
	train: 0.999802945957929 18.224498258260148 	test: 0.9995402298850574 12.47617277712841
retrain  6 :
	train: 0.9998522094684468 14.893285881632314 	test: 0.9995977011494253 11.167382208116138
retrain  7 :
	train: 0.9998522094684468 12.179236018362564 	test: 0.9996551724137931 10.191325855723528
retrain  8 :
	train: 0.9998768412237056 9.91499335813508 	test: 0.9996551724137931 9.432963169464724
retrain  9 :
	train: 0.9999261047342234 8.048439057197376 	test: 0.9997126436781609 8.83806400051397
retrain  10 :
	train: 0.9999261047342234 6.578138417174225 	test: 0.999712643678

KeyboardInterrupt: 

In [50]:
gtgp.print_model()

print(classification_report(y_train,np.argmax(gtgp.train_p,axis=1),digits=5))
print(classification_report(y_test,np.argmax(gtgp.test_p,axis=1),digits=5))

----------------GTGP-------------
Number of Trees: 103
Average of depth: 1.4951456310679612
Number of nodes: 417
              precision    recall  f1-score   support

           0    1.00000   1.00000   1.00000     31910
           1    1.00000   1.00000   1.00000        34
           2    1.00000   1.00000   1.00000       120
           3    1.00000   1.00000   1.00000      6231
           4    1.00000   1.00000   1.00000      2287
           5    1.00000   1.00000   1.00000         7
           6    1.00000   1.00000   1.00000         9

    accuracy                        1.00000     40598
   macro avg    1.00000   1.00000   1.00000     40598
weighted avg    1.00000   1.00000   1.00000     40598

              precision    recall  f1-score   support

           0    1.00000   0.99993   0.99996     13676
           1    1.00000   1.00000   1.00000        15
           2    0.98077   1.00000   0.99029        51
           3    1.00000   0.99963   0.99981      2671
           4    1.0

In [51]:
gtgp.print_model()

print(classification_report(y_train,np.argmax(gtgp.train_p,axis=1),digits=5))
print(classification_report(y_test,np.argmax(gtgp.test_p,axis=1),digits=5))

----------------GTGP-------------
Number of Trees: 103
Average of depth: 1.4951456310679612
Number of nodes: 417
              precision    recall  f1-score   support

           0    1.00000   1.00000   1.00000     31910
           1    1.00000   1.00000   1.00000        34
           2    1.00000   1.00000   1.00000       120
           3    1.00000   1.00000   1.00000      6231
           4    1.00000   1.00000   1.00000      2287
           5    1.00000   1.00000   1.00000         7
           6    1.00000   1.00000   1.00000         9

    accuracy                        1.00000     40598
   macro avg    1.00000   1.00000   1.00000     40598
weighted avg    1.00000   1.00000   1.00000     40598

              precision    recall  f1-score   support

           0    1.00000   0.99993   0.99996     13676
           1    1.00000   1.00000   1.00000        15
           2    0.98077   1.00000   0.99029        51
           3    1.00000   0.99963   0.99981      2671
           4    1.0

# Decision Tree

In [11]:
from sklearn.tree import DecisionTreeClassifier

In [15]:
for i in range(30):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
    y_train,y_test,y_train_one_hot,y_test_one_hot = to_one_hot(y_train,y_test)
    clf = DecisionTreeClassifier()
    clf.fit(X_train,y_train)

    num_trees = 1
    depth = clf.tree_.max_depth
    num_nodes = clf.tree_.node_count

    train_acc = accuracy_score(y_train,clf.predict(X_train))
    test_acc = accuracy_score(y_test,clf.predict(X_test))
    train_f1 = f1_score(y_train,clf.predict(X_train),average='macro')
    test_f1 = f1_score(y_test,clf.predict(X_test),average='macro')

    with open('./benchmark_DC/'+dataset+'.csv','a') as f:
        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)

# XGBoost

In [16]:
import xgboost

In [30]:
for i in range(30):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
    y_train,y_test,y_train_one_hot,y_test_one_hot = to_one_hot(y_train,y_test)
    
    # xgb = xgboost.XGBClassifier(min_child_weight=0,max_depth=3,n_estimators=100)
    # xgb = xgboost.XGBClassifier(min_child_weight=0,max_depth=4)
    xgb = xgboost.XGBClassifier(n_estimators=10000)
    xgb.fit(X_train,y_train)

    import json

    def item_generator(json_input, lookup_key):
        if isinstance(json_input, dict):
            for k, v in json_input.items():
                if k == lookup_key:
                    yield v
                else:
                    yield from item_generator(v, lookup_key)
        elif isinstance(json_input, list):
            for item in json_input:
                yield from item_generator(item, lookup_key)

    def tree_depth(json_text):
        json_input = json.loads(json_text)
        depths = list(item_generator(json_input, 'depth'))
        return max(depths) + 1 if len(depths) != 0 else 1

    train_acc = accuracy_score(y_train,xgb.predict(X_train))
    test_acc = accuracy_score(y_test,xgb.predict(X_test))
    train_f1 = f1_score(y_train,xgb.predict(X_train),average='macro')
    test_f1 = f1_score(y_test,xgb.predict(X_test),average='macro')

    booster = xgb.get_booster()

    tree_df = booster.trees_to_dataframe()
    depths = [tree_depth(x) for x in booster.get_dump(dump_format = "json")]
    num_trees = len(depths)
    depth = np.average(depths)
    num_nodes = len(tree_df)

    with open('./benchmark_xgb/'+dataset+'.csv','a') as f:
        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)

KeyboardInterrupt: 

# GDBT

In [18]:
from sklearn.ensemble import GradientBoostingClassifier

In [19]:
for i in range(30):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
    y_train,y_test,y_train_one_hot,y_test_one_hot = to_one_hot(y_train,y_test)
    clf = GradientBoostingClassifier()
    # clf = GradientBoostingClassifier()
    clf.fit(X_train,y_train)

    train_acc = accuracy_score(y_train,clf.predict(X_train))
    test_acc = accuracy_score(y_test,clf.predict(X_test))
    train_f1 = f1_score(y_train,clf.predict(X_train),average='macro')
    test_f1 = f1_score(y_test,clf.predict(X_test),average='macro')

    num_trees = len([ est for ests in clf.estimators_ for est in ests])
    depth = np.average([ max(1,est.tree_.max_depth) for ests in clf.estimators_ for est in ests])
    num_nodes = sum([ est.tree_.node_count for ests in clf.estimators_ for est in ests])

    with open('./benchmark_GBDT/'+dataset+'.csv','a') as f:
        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)

# Random Forest

In [20]:
from sklearn.ensemble import RandomForestClassifier

In [21]:
for i in range(30):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
    y_train,y_test,y_train_one_hot,y_test_one_hot = to_one_hot(y_train,y_test)
    rfc = RandomForestClassifier()
    rfc.fit(X_train,y_train)

    train_acc = accuracy_score(y_train,rfc.predict(X_train))
    test_acc = accuracy_score(y_test,rfc.predict(X_test))
    train_f1 = f1_score(y_train,rfc.predict(X_train),average='macro')
    test_f1 = f1_score(y_test,rfc.predict(X_test),average='macro')

    num_trees = len(rfc.estimators_)
    depth = np.average([est.tree_.max_depth for est in rfc.estimators_])
    num_nodes = sum([est.tree_.node_count for est in rfc.estimators_])

    with open('./benchmark_RF/'+dataset+'.csv','a') as f:
        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)